In [105]:
import math
import random
from copy import copy

K = 64
BANDWIDTH = 5_000_000_000

nodeList, edgeList = [], []

core = int(math.pow(K, 2) // 4)
agg = int(math.pow(K, 2) // 2)
access = int(math.pow(K, 2) // 2)

print(f"core: {core} agg: {agg} access: {access}")

core: 1024 agg: 2048 access: 2048


In [106]:
# ASSIGN IDS TO CORE, AGGREGATION AND ACCESS SWITCHES

core_switches, agg_switches, access_switches = [], [], []

for id_core in range(0, core):
    core_switches.append({
        "id": id_core,
        "portList": [],
        "deviceLevel": "core",
    })

for id_agg in range(id_core+1, id_core+agg+1):
    agg_switches.append({
        "id": id_agg,
        "portList": [],
        "deviceLevel": "aggregation",
    })

for id_access in range(id_agg+1, id_agg+access+1):
    access_switches.append({
        "id": id_access,
        "portList": [],
        "deviceLevel": "access",
    })

nodeList = core_switches + agg_switches + access_switches

print(len(core_switches), len(agg_switches), len(access_switches))

1024 2048 2048


In [107]:
# add port information to node dict

for node in nodeList:
    for port_id in range(K):
        node['portList'].append({
            'id': f'port{port_id}',
            'bw': BANDWIDTH,
        })
        

In [108]:
#   UTILITY FUNCTIONS FOR TOPO GENERATION

def sample_with_removal(ar, k):
    s = random.sample(ar, k)
    ar = [x for x in ar if x not in s]
    return ar, s

def get_next_unused_port(node):
    for port in node['portList']:
        if port['conn'] is None:
            return port['id']
    return None

def split_list(lst, n):
    """Split a list into n equal-sized parts."""
    if n <= 0:
        raise ValueError("Number of parts must be greater than 0")
    elif n > len(lst):
        raise ValueError("Number of parts cannot exceed the length of the list")

    # Calculate the size of each part
    part_size = len(lst) // n
    # Calculate the remainder to distribute if the list size is not divisible evenly
    remainder = len(lst) % n

    # Generate the parts
    parts = [lst[i * part_size + min(i, remainder):(i + 1) * part_size + min(i + 1, remainder)] for i in range(n)]

    return parts


In [109]:
pod_access, pod_agg = copy(access_switches), copy(agg_switches)

pods = []

for pod_id in range(K):
    # radomly assign access and aggregation nodes into pods 
    pod_access, acc_nodes = sample_with_removal(pod_access, K//2)
    pod_agg, agg_nodes = sample_with_removal(pod_agg, K//2)
    
    pods.append({
        "id": pod_id,
        "agg_nodes": agg_nodes,
        "acc_nodes": acc_nodes
    })

    # each aggregation node will connect to 2 random core nodes
    core_conn_port_id = pod_id # on the core switch the connection will happen on the port for the respectivde pod

    '''
        Example: 
            pods [a,b] [c d] [e f] [g h] # ignoring access siwtches in the 2 pods
            coreswitchs [A B C D]

            connections: node:port
                a:0 -> A:0 
                a:1 -> B:0
                b:0 -> C:0
                b:1 -> D:0

                c:0 -> A:1
                c:1 -> B:1
                d:0 -> C:1
                d:1 -> D:1
    '''

    core_switches_per_agg_node = split_list(core_switches, len(agg_nodes))

    for idx, (agg_node, core_conns) in enumerate(zip(agg_nodes, core_switches_per_agg_node)):
        # print(idx, agg_node['id'], [x['id'] for x in core_conns])
        
        for core_switch in core_conns:
            src, dest = agg_node['id'], core_switch['id']
            src_p = agg_node['portList'][idx]['id']
            dest_p = core_switch['portList'][core_conn_port_id]['id']

            edge = {
                "srcNode": src,
                "destNode": dest,
                "srcPort": src_p,
                "destPort": dest_p
            }

            reverse = {
                "srcNode": dest,
                "destNode": src,
                "srcPort": dest_p,
                "destPort": src_p
            }

            edgeList.append(edge)
            edgeList.append(reverse)

        '''
            The intra pod connections are simpler:
                The aggragation nodes use ports starting from K//2 to K (0->k//2 are used for connections to core switches)
                The access switches use ports from 0->K//2 and reserve ports from K//2->K for connecting to devices
        '''
        
        for idx, access_switch in enumerate(acc_nodes):
            agg_offset = K // 2

            src, dest = agg_node['id'], access_switch['id']

            src_p = agg_node['portList'][idx+agg_offset]['id']
            dest_p = access_switch['portList'][idx]['id']

            edge = {
                "srcNode": src,
                "destNode": dest,
                "srcPort": src_p,
                "destPort": dest_p
            }

            reverse = {
                "srcNode": dest,
                "destNode": src,
                "srcPort": dest_p,
                "destPort": src_p
            }

            edgeList.append(edge)
            edgeList.append(reverse)
            
len(nodeList), len(edgeList)

(5120, 262144)

In [103]:
import json

with open(f"data/fat_tree_{K}_{BANDWIDTH}.json", 'w+') as f:
    json.dump(
        {'nodeList': nodeList, 'edgeList': edgeList},
        f,
        indent=4
    )